## 月別・プロダクト別の売上を集計した特徴量テーブル

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.default.feature_sales_by_product AS
SELECT
  so.SALESORG AS REGION,
  soi.PRODUCTID,
  CONCAT(SUBSTRING(CAST(so.CREATEDAT AS STRING), 1, 4), '-', LPAD(SUBSTRING(CAST(so.CREATEDAT AS STRING), 5, 2), 2, '0')) AS FISCAL_YEAR_MONTH,
  SUM(soi.NETAMOUNT) AS TOTAL_NETAMOUNT
FROM
  workspace.default.bronze_salesorders so
JOIN
  workspace.default.bronze_salesorderitems soi
ON
  so.SALESORDERID = soi.SALESORDERID
GROUP BY
  so.SALESORG,
  soi.PRODUCTID,
  CONCAT(SUBSTRING(CAST(so.CREATEDAT AS STRING), 1, 4), '-', LPAD(SUBSTRING(CAST(so.CREATEDAT AS STRING), 5, 2), 2, '0'))
ORDER BY
  soi.PRODUCTID,
  FISCAL_YEAR_MONTH

In [0]:
%sql
SELECT * FROM workspace.default.feature_sales_by_product;

## 地域別の売上予測を返す関数

In [0]:
%sql
CREATE OR REPLACE FUNCTION workspace.default.sales_prediction(start_month STRING, end_month STRING)
RETURNS TABLE (
  REGION STRING,
  FISCAL_YEAR_MONTH STRING,
  predicted_TOTAL_NETAMOUNT DOUBLE,
  predicted_TOTAL_NETAMOUNT_lower DOUBLE,
  predicted_TOTAL_NETAMOUNT_upper DOUBLE,
  predicted_at TIMESTAMP
)
COMMENT 'この関数は、今後3ヶ月の地域別の売上予測を行います。'
RETURN
  SELECT
  REGION,
  date_format(to_date(FISCAL_YEAR_MONTH, 'yyyyMM'), 'yyyy-MM') AS FISCAL_YEAR_MONTH,
  predicted_TOTAL_NETAMOUNT,
  predicted_TOTAL_NETAMOUNT_lower,
  predicted_TOTAL_NETAMOUNT_upper,
  predicted_at
  -- ここに自身で予測した結果のテーブル名を入れる
  -- FROM workspace.default.forecast_predictions_1754378418599
  WHERE date_format(to_date(FISCAL_YEAR_MONTH, 'yyyyMM'), 'yyyy-MM') BETWEEN start_month AND end_month
;

## 指定された期間の月別・プロダクト別の売上のテーブルを返す関数

In [0]:
%sql
CREATE OR REPLACE FUNCTION workspace.default.sales_report(start_month STRING, end_month STRING)
RETURNS TABLE (
  REGION STRING,
  PRODUCTID STRING,
  FISCAL_YEAR_MONTH STRING,
  TOTAL_NETAMOUNT DOUBLE
)
COMMENT '指定された開始月と終了月の期間における、地域・商品ごとの月次売上レポートを返す。'
RETURN
  SELECT
    REGION,
    PRODUCTID,
    FISCAL_YEAR_MONTH,
    TOTAL_NETAMOUNT
  FROM
    workspace.default.feature_sales_by_product
  WHERE
    FISCAL_YEAR_MONTH BETWEEN start_month AND end_month
;

In [0]:
%sql
-- 指定された開始月(start_month)と終了月(end_month)の期間における、
-- 地域(REGION)ごと、商品(PRODUCTID)ごとの月次売上(TOTAL_NETAMOUNT)を
-- workspace.default.feature_sales_by_productテーブルから抽出し、レポートとして返す関数sales_reportを生成しなさい

## 作成した関数のテスト

In [0]:
%sql
SELECT * FROM workspace.default.sales_report("2019-01", "2019-03");

SELECT * FROM workspace.default.sales_prediction("2019-07", "2019-08");